In [ ]:

##Gage Black
##Homework 6
##ECGR 4105
#Single and Multi

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
data_path = '../data-unversioned'
cirfar10 = datasets.CIFAR10(data_path, train=True, download=True)
cirfar10_val = datasets.CIFAR10(data_path, train=False, download=True)

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

data_path = '../data-unversioned/p1ch7/'
cifar10 = cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
    ]))



In [ ]:
##Ripped from files10 CIFARS

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
    ]))

label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label]) for img, label in cifar10 if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in cifar10_val if label in [0, 2]]

In [ ]:
##Load the trainer
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
##Single Sequences
seqSingle = nn.Sequential(
  nn.Linear(3072, 512),
  nn.Tanh(),
  nn.Linear(512, 10),
  nn.LogSoftmax(dim=1))



In [ ]:
##Multiple Sequences



seqMulti = nn.Sequential(
  nn.Linear(3072, 512),
  nn.Tanh(),
  nn.Linear(512, 256),
  nn.Tanh(),
  nn.Linear(256, 128),
  nn.Tanh(),
  nn.Linear(128, 10),
  nn.LogSoftmax(dim=1))

learning_rate = 0.01
nEpochs = 300
loss_fn = nn.NLLLoss()

optimizer_single = optim.SGD(seqSingle.parameters(), lr=learning_rate)
optimizer_multi = optim.SGD(seqMulti.parameters(), lr=learning_rate)

In [20]:
##EPoch in range of multi steps
for epoch in range(1, nEpochs + 1):
  for imgs, labels in train_loader:

    batch_size = imgs.shape[0]
    outputs = seqMulti(imgs.view(batch_size, -1))
    loss = loss_fn(outputs, labels)

    optimizer_multi.zero_grad()
    loss.backward()
    optimizer_multi.step()

  if ((epoch == 1) or (epoch % 10 == 0)):
    print(f"Epoch {epoch}, Loss {loss.item():.4f}")

Epoch 1, Loss 1.6024
Epoch 10, Loss 1.7445
Epoch 20, Loss 1.1537
Epoch 30, Loss 0.4073
Epoch 40, Loss 0.0967
Epoch 50, Loss 0.1502
Epoch 60, Loss 0.1003
Epoch 70, Loss 0.0168
Epoch 80, Loss 0.0033
Epoch 90, Loss 0.0016
Epoch 100, Loss 0.0013
Epoch 110, Loss 0.0021
Epoch 120, Loss 0.0010
Epoch 130, Loss 0.0008
Epoch 140, Loss 0.0011
Epoch 150, Loss 0.0012
Epoch 160, Loss 0.0011
Epoch 170, Loss 0.0012
Epoch 180, Loss 0.0006
Epoch 190, Loss 0.0005
Epoch 200, Loss 0.0007
Epoch 210, Loss 0.0009
Epoch 220, Loss 0.0005
Epoch 230, Loss 0.0010
Epoch 240, Loss 0.0004
Epoch 250, Loss 0.0004
Epoch 260, Loss 0.0005
Epoch 270, Loss 0.0004
Epoch 280, Loss 0.0005
Epoch 290, Loss 0.0003
Epoch 300, Loss 0.0003


In [21]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
  for imgs, labels in train_loader:
    outputs = seqSingle(imgs.view(imgs.shape[0], -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += int((predicted == labels).sum())

print("Single Layer Train Acc Percentage: %f" % (correct / total))

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
  for imgs, labels in val_loader:
    outputs = seqSingle(imgs.view(imgs.shape[0], -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += int((predicted == labels).sum())

print("Single Layer Vali Acc Percentage: %f" % (correct / total))

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
  for imgs, labels in train_loader:
    outputs = seqMulti(imgs.view(imgs.shape[0], -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += int((predicted == labels).sum())

print("Multi Layer Train Acc Percentage: %f" % (correct / total))

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
  for imgs, labels in val_loader:
    outputs = seqMulti(imgs.view(imgs.shape[0], -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += int((predicted == labels).sum())

print("Multi Layer Valid Acc Percentage: %f" % (correct / total))


Single Layer Train Acc Percentage: 0.901880
Single Layer Vali Acc Percentage: 0.469100
Multi Layer Train Acc Percentage: 1.000000
Multi Layer Valid Acc Percentage: 0.452100
